In [24]:
print_Invest = True

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date

Необходимые для работы функции:

In [2]:
month_map = {'ЯНВ': '01', 'ФЕВ': '02', 'МАР': '03', 'АПР': '04', 'МАЙ': '05', 
             'ИЮН': '06', 'ИЮЛ': '07', 'АВГ': '08', 'СЕН': '09', 'ОКТ': '10', 
             'НОЯ': '11', 'ДЕК': '12'}

files_list = [x for x in os.listdir() if 'Прогноз_' in x and '~' not in x]
map_list = []
for file in files_list:
    data = pd.read_excel(file, sheet_name='snapshot')
    data_list = data.iloc[13:, 1].unique().tolist()
    map_list = map_list + data_list

map_set = list(set(map_list))
map_df = pd.DataFrame({'ПВК': map_set}, index= [x for x in range(len(map_set))])

data_did = pd.read_excel('Currency.xlsx')
data_did = data_did.iloc[:, 1:].reset_index(drop=True)
CCY_dict = {x:y for x,y in zip(data_did.iloc[:, 0].tolist(), data_did.iloc[:, 1].tolist())}

inter_list = list(set(map_df.iloc[:, 0]).intersection(set(data_did.iloc[:, 0])))
manual_df = map_df[~map_df.iloc[:, 0].isin(inter_list)]

In [8]:
manual_df
# manual_df.to_excel('Currency_map.xlsx')

,ПВК
5,Общий результат
8,Не присвоено


Рабочие данные:

In [16]:
files_list = os.listdir('.')
work_files_list = [x for x in files_list if ('.xlsm' in x and '~' not in x and 'Шапка' not in x)]
SUEK_work_files_list = [x for x in work_files_list if 'С' in x]
EURO_work_files_list = [x for x in work_files_list if 'Е' in x]

data_work_files_list = [SUEK_work_files_list, EURO_work_files_list]

In [17]:
work_files_list

['Прогноз_ЕХ_(2024_03_12).xlsm',
 'Прогноз_ЕХ_КЗ_(2024_03_12).xlsm',
 'Прогноз_СГК_(2024_03_12).xlsm',
 'Прогноз_СУЭК_(2024_03_12).xlsm']

In [11]:
Data_for_header_list = ['Шапка_ЕХ.xlsm', 'Шапка_СУЭК.xlsm']
Sheet_in_data_for_header = 'result'

for Group_file_for_header in Data_for_header_list:
    
    Group_data_for_header = pd.read_excel(Group_file_for_header, sheet_name=Sheet_in_data_for_header)

    # Проверка на соответствие числа столбцов 28-и
    while Group_data_for_header.shape[1] >28:
        Group_data_for_header = Group_data_for_header.drop(Group_data_for_header.columns[-1], axis=1)

    # Создание header
    Group_start_columns_list = Group_data_for_header.columns.tolist()
    Group_first_row = pd.Series({x:y for x,y in zip(Group_start_columns_list, Group_start_columns_list)})
    Group_header = Group_data_for_header.iloc[:4]
    Group_header.loc[Group_header.shape[0]] = Group_first_row
    Group_header = Group_header.iloc[[-1,0,1,2,3]]

    # проверка на Group и присваивание переменных
    if 'С' in Group_file_for_header:
        SUEK_header = Group_header
    elif 'Е' in Group_file_for_header:
        EURO_header = Group_header
    else:
        assert 1 != 1

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14348\2198211952.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Group_header.loc[Group_header.shape[0]] = Group_first_row
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14348\2198211952.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Group_header.loc[Group_header.shape[0]] = Group_first_row


In [19]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping

In [20]:
SUEK_cocat_data = pd.DataFrame()
EURO_cocat_data = pd.DataFrame()

for Group_work_file in work_files_list:

    # Чтение файла
    Group_data = pd.read_excel(Group_work_file, sheet_name='snapshot')
    Group_data_work = Group_data

    # Удаление лишних строк
    Group_data_work = Group_data_work.iloc[13:].reset_index(drop=True)

    # Проверка на соответсвие кол-ва столбцов 20-и
    while Group_data_work.shape[1] >20:
        Group_data_work = Group_data_work.drop(Group_data_work.columns[-1], axis=1)

    # Добавление значений в столбец Дата
    Group_data_work['Дата'] = Group_data_work.iloc[:, 2] + '-' + Group_data_work.iloc[:, 3].replace(month_map) + '-01'

    # Merge и добавление столбца CompCode
    # merge_df = Group_data_work.merge(SalesUnits_work_data, right_on='sapId', left_on='Unnamed: 4', how='left')['id']
    Group_data_work['CompCode'] = merge_SalesUnits(Group_data_work, col='Unnamed: 4', id_col='sapId', merge_col='id')

    # Map названия валют
    Group_data_work.iloc[:, 1] = Group_data_work.iloc[:, 1].replace(CCY_dict)
    
    if Group_work_file in SUEK_work_files_list:
        
        Group_data_work[['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']] = Group_data_work[['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']].fillna(0)
        Group_data_work['Незаконтрактовано_035'] = Group_data_work['Unnamed: 6'] + Group_data_work['Unnamed: 7'] + Group_data_work['Unnamed: 8']
        Group_data_work['Законтрактовано_035'] = Group_data_work['Unnamed: 9']

        Group_data_work[['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']] = Group_data_work[['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']].fillna(0)
        Group_data_work['Незаконтрактовано_075'] = Group_data_work['Unnamed: 13'] + Group_data_work['Unnamed: 14'] + Group_data_work['Unnamed: 15']
        Group_data_work['Законтрактовано_075'] = Group_data_work['Unnamed: 16']

        Group_data_work[['Незаконтрактовано', 'Законтрактовано', 'Сумма']] = 0

        # Замена пустых строк на нули
        Group_data_work[['Unnamed: 10','Unnamed: 11', 'Unnamed: 17']] = Group_data_work[['Unnamed: 10','Unnamed: 11', 'Unnamed: 17']].fillna(0)

        # Добавление значений в столбец Итого_законтрактовано по условию
        Group_data_work.loc[Group_data_work['Unnamed: 17'] > (Group_data_work['Unnamed: 10'] + Group_data_work['Unnamed: 11']), 'Законтрактовано'] = Group_data_work['Законтрактовано_075']
        Group_data_work.loc[~(Group_data_work['Unnamed: 17'] > (Group_data_work['Unnamed: 10'] + Group_data_work['Unnamed: 11'])), 'Законтрактовано'] = Group_data_work['Законтрактовано_035']

        
        # Порядок столбцов
        Group_data_work = Group_data_work[['CompCode', 'Unnamed: 0','Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4','Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9','Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12','Незаконтрактовано_035', 'Законтрактовано_035', 'Unnamed: 13','Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17','Unnamed: 18', 'Незаконтрактовано_075', 'Законтрактовано_075','Незаконтрактовано', 'Законтрактовано', 'Сумма', 'Дата']]

        # проверка на 28-м столбцов
        assert Group_data_work.shape[1] == 28

        # Объединение данных для всего суэк
        SUEK_cocat_data = pd.concat([SUEK_cocat_data, Group_data_work], axis=0)
    
    if Group_work_file in EURO_work_files_list:
    
        Group_data_work[['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']] = Group_data_work[['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']].fillna(0)
        Group_data_work['Незаконтрактовано_075'] = Group_data_work['Unnamed: 6'] + Group_data_work['Unnamed: 7'] + Group_data_work['Unnamed: 8']
        Group_data_work['Законтрактовано_075'] = Group_data_work['Unnamed: 9']

        Group_data_work[['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 12']] = Group_data_work[['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 12']].fillna(0)
        Group_data_work['Незаконтрактовано_085'] = Group_data_work['Unnamed: 13'] + Group_data_work['Unnamed: 14'] + Group_data_work['Unnamed: 12']
        Group_data_work['Законтрактовано_085'] = Group_data_work['Unnamed: 15']

        Group_data_work[['Незаконтрактовано', 'Законтрактовано', 'Сумма']] = 0

        # Замена пустых строк на нули
        Group_data_work[['Unnamed: 16','Unnamed: 17', 'Unnamed: 10']] = Group_data_work[['Unnamed: 16','Unnamed: 17', 'Unnamed: 10']].fillna(0)

        # Добавление значений в столбец Итого_законтрактовано по условию
        Group_data_work.loc[Group_data_work['Unnamed: 10'] > (Group_data_work['Unnamed: 16'] + Group_data_work['Unnamed: 17']), 'Законтрактовано'] = Group_data_work['Законтрактовано_075']
        Group_data_work.loc[~(Group_data_work['Unnamed: 10'] > (Group_data_work['Unnamed: 16'] + Group_data_work['Unnamed: 17'])), 'Законтрактовано'] = Group_data_work['Законтрактовано_085']

        # Порядок столбцов
        Group_data_work = Group_data_work[['CompCode', 'Unnamed: 0','Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4','Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9','Unnamed: 10', 'Unnamed: 11', 'Незаконтрактовано_075', 'Законтрактовано_075','Unnamed: 12', 'Unnamed: 13','Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17','Unnamed: 18', 'Незаконтрактовано_085', 'Законтрактовано_085','Незаконтрактовано', 'Законтрактовано', 'Сумма', 'Дата']]

        # проверка на 28-м столбцов
        assert Group_data_work.shape[1] == 28

        # Объединение данных для всего суэк
        EURO_cocat_data = pd.concat([EURO_cocat_data, Group_data_work], axis=0)


# Названия столбцов сравнять 
SUEK_header.columns = SUEK_cocat_data.columns
EURO_header.columns = EURO_cocat_data.columns
# Добавление headers
SUEK_to_print = pd.concat([SUEK_header, SUEK_cocat_data], axis=0)
EURO_to_print = pd.concat([EURO_header, EURO_cocat_data], axis=0)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14348\3193353353.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Group_data_work[['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']] = Group_data_work[['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']].fillna(0)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14348\3193353353.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in 

In [22]:
EURO_to_print
SUEK_to_print

,CompCode,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Незаконтрактовано_075,Законтрактовано_075,Незаконтрактовано,Законтрактовано,Сумма,Дата
4,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Версия,035,Unnamed: 8,Unnamed: 9,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Освоение. Прогноз,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Сумма прогноза, USD",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,Тип прогноз. данных,Резервирование,Заявка на закупку,Запрос (док 19*),...,Запрос (док 19*),Заказ на поставку,Факт,Результат,Незаконтрактовано,Законтрактовано,Незаконтрактовано,Законтрактовано,Сумма,NaN
3,NaN,NaN,ПВК,Календарный год,Календарный месяц,Балансовая единица,NaN,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,6700,NaN,#,2024,НОЯ,6700,"АО ""Ургалуголь""",0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,2024-11-01
1361,2100,NaN,#,2024,ДЕК,2100,"АО ""СУЭК-Красноярск""",0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,2024-12-01
1362,3200,NaN,#,2024,ДЕК,3200,"АО ""Разрез Харанорский""",0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,2024-12-01
1363,6800,NaN,#,2024,ДЕК,6800,"ООО СК ""Малый порт""",0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,2024-12-01


In [25]:
Output_file_SUEK = "_".join([str(date.today()), 'SUEK_Invest.xlsx'])
Output_file_EURO = "_".join([str(date.today()), 'EUROCHEM_Invest.xlsx'])
Sheet_in_output_file = 'Invest'

if print_Invest == True:
    SUEK_to_print.to_excel(Output_file_SUEK, sheet_name=Sheet_in_output_file, index=False, header=False)
    EURO_to_print.to_excel(Output_file_EURO, sheet_name=Sheet_in_output_file, index=False, header=False)